In [6]:
import json
from pathlib import Path
with open(Path("../results/hotpotqa_samples.json"), "r") as f:
    hotpotqa_samples = json.load(f)
with open(Path("../results/wiki_samples.json"), "r") as f:
    wiki_samples = json.load(f)
with open(Path("../results/gpt-5_explicit_injection_tasks.json"), "r") as f:
    explicit_injection_tasks = json.load(f)
with open(Path("../results/gpt-5_subtle_injection_tasks.json"), "r") as f:
    subtle_injection_tasks = json.load(f)

In [19]:
import pandas as pd
from tqdm import tqdm
import random

random.seed(42)

def create_examples():
    examples = []
    
    for i in tqdm(range(len(hotpotqa_samples)), desc="HotPotQA"):
        data = hotpotqa_samples[i]
        
        examples.append({
            "context": data['context'],
            "primary_task": f"Answer the following question based on the context: {data['question']}",
            "secondary_task": ""
        })
    
    primary_tasks = [
        "Summarize the following text in 2-3 sentences.",
        "Analyze the sentiment of the following text and classify it as positive, negative, or neutral.",
        "Identify and list all named entities (people, organizations, locations) in the following text.",
        "Rephrase the following text while preserving its original meaning.",
        "Categorize the following text into the most appropriate topic or genre.",
        "Extract the key facts and details from the following text.",
        "Continue or complete the following text in a coherent manner.",
        "Read the following text and determine if the claims made are factually accurate based on the context provided.",
        "Provide a detailed analysis and understanding of the following text."
    ]
    
    for i in tqdm(range(len(wiki_samples)), desc="Wikipedia"):
        data = wiki_samples[i]
        task_text = random.choice(primary_tasks)
        
        examples.append({
            "context": data,
            "primary_task": task_text,
            "secondary_task": ""
        })
    
    return examples

print("Creating examples...")
all_examples = create_examples()
examples = random.sample(all_examples, min(2250, len(all_examples)))
print(f"✓ Created {len(examples)} examples from {len(all_examples)} total")

df = pd.DataFrame(examples)
df.to_csv("../results/sampled_examples.csv", index=False)
print("✓ Saved sampled examples")

Creating examples...


Wikipedia: 100%|██████████| 1500/1500 [00:00<00:00, 2450898.32it/s]

✓ Created 2250 examples from 2251 total
✓ Saved sampled examples


In [ ]:
import re

def split_into_sentences(text):
    sentence_endings = r'(?<=[.!?])\s+(?=[A-Z])'
    sentences = re.split(sentence_endings, text)
    return [s.strip() for s in sentences if s.strip()]

def inject_task_between_sentences(context, injection_task):
    sentences = split_into_sentences(context)
    
    if len(sentences) <= 2:
        return f"{context} {injection_task}"
    
    injection_position = random.randint(1, len(sentences) - 1)
    
    injected_context = ' '.join(sentences[:injection_position]) + ' ' + injection_task + ' ' + ' '.join(sentences[injection_position:])
    
    return injected_context.strip()

all_injections = explicit_injection_tasks + subtle_injection_tasks
print(f"Total injection tasks available: {len(all_injections)}")

clean_examples = []
corrupted_examples = []

print("\nCreating clean and corrupted datasets...")

for idx in tqdm(range(len(examples))):
    example = examples[idx]
    
    if idx < len(examples) // 2:
        clean_examples.append({
            "label": "clean",
            "primary_task": example["primary_task"],
            "context": example["context"],
            "secondary_task": "",
            "injection_type": "none",
            "full_prompt": f"{example['primary_task']}\n\n# Context: {example['context']}"
        })
    else:
        injection_task = random.choice(all_injections)
        is_subtle = injection_task in subtle_injection_tasks
        
        corrupted_context = inject_task_between_sentences(example["context"], injection_task)
        
        corrupted_examples.append({
            "label": "corrupted",
            "primary_task": example["primary_task"],
            "context": corrupted_context,
            "secondary_task": injection_task,
            "injection_type": "subtle" if is_subtle else "explicit",
            "full_prompt": f"{example['primary_task']}\n\n# Context: {corrupted_context}"
        })

print(f"\n✓ Clean examples: {len(clean_examples)}")
print(f"✓ Corrupted examples: {len(corrupted_examples)}")

Total injection tasks available: 800

Creating clean and corrupted datasets...


100%|██████████| 2250/2250 [00:00<00:00, 89971.34it/s]


✓ Clean examples: 1125
✓ Corrupted examples: 1125


In [21]:
print("Combining and shuffling final dataset...")
final_dataset = clean_examples + corrupted_examples
random.shuffle(final_dataset)

df_final = pd.DataFrame(final_dataset)

print(f"\n✓ Total examples: {len(df_final)}")
print(f"✓ Clean: {len(df_final[df_final['label'] == 'clean'])}")
print(f"✓ Corrupted: {len(df_final[df_final['label'] == 'corrupted'])}")

df_final.to_csv("../results/final_dataset.csv", index=False)

print("\n=== Dataset Summary ===")
print(f"Total rows: {len(df_final)}")
print(f"Balanced: {len(df_final[df_final['label'] == 'clean'])} clean, {len(df_final[df_final['label'] == 'corrupted'])} corrupted")
print(f"\nInjection breakdown in corrupted:")
corrupted_df = df_final[df_final['label'] == 'corrupted']
print(f"  Explicit: {len(corrupted_df[corrupted_df['injection_type'] == 'explicit'])}")
print(f"  Subtle: {len(corrupted_df[corrupted_df['injection_type'] == 'subtle'])}")

print("\n✅ Dataset generation complete!")


Combining and shuffling final dataset...

✓ Total examples: 2250
✓ Clean: 1125
✓ Corrupted: 1125

=== Dataset Summary ===
Total rows: 2250
Balanced: 1125 clean, 1125 corrupted

Injection breakdown in corrupted:
  Explicit: 540
  Subtle: 585

✅ Dataset generation complete!
